In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from ebooklib import epub
import datetime

In [4]:
request = requests.get(r'https://www.lesechos.fr/')

In [5]:
soup = BeautifulSoup(request.text, "html.parser")

In [6]:
links_with_text = []
for a in soup.find_all('a', href=True): 
    if a.text: 
        links_with_text.append(a['href'])

In [7]:
df_echo = pd.DataFrame(columns=["href","id", "Label", "Sublabel", "Title", "Lead", "Publication", "Description"])

In [8]:
df_echo["href"] = links_with_text

In [9]:
def split_it(href):
    try:
        result = re.findall('\d{7}$', href)
        result = result[0]
    except:
        result = ""
    return result

In [10]:
df_echo["id"] = df_echo["href"].apply(lambda x: split_it(x))

In [11]:
df_echo = df_echo.loc[df_echo["id"]!=""].reset_index(drop=True)

In [12]:
r = requests.get(r'https://api.lesechos.fr/api/v1/articles/1242262')
r = r.json()

In [13]:
for i, val in enumerate(df_echo.itertuples()):
    try:
        url = r"https://api.lesechos.fr/api/v1/articles/" + str(df_echo.at[i, "id"])
        r =  requests.get(url).json()
        df_echo.at[i, "Label"] = r["stripes"][0]["mainContent"][0]["data"]["section"]["label"] # Label
        df_echo.at[i, "Sublabel"]= r["stripes"][0]["mainContent"][0]["data"]["subsection"]["label"] # Sublabel
        df_echo.at[i, "Title"]= r["stripes"][0]["mainContent"][0]["data"]["title"] # Title
        df_echo.at[i, "Lead"]= r["stripes"][0]["mainContent"][0]["data"]["lead"] # Lead
        df_echo.at[i, "Publication"]= r["stripes"][0]["mainContent"][0]["data"]["publicationDate"]
        df_echo.at[i, "Description"] = r["stripes"][0]["mainContent"][0]["data"]["description"] # Description
        
    except:
        print("KO " + str(df_echo.at[i,"id"]))
       

KO 1243470
KO 1243457
KO 1243380


In [14]:
df_echo = df_echo.loc[df_echo["Label"] != ""].reset_index(drop=True)

In [15]:
# Filtre sur les articles du jours
now = datetime.date.today()
df_echo["Publication"] = pd.to_datetime(df_echo["Publication"])
df_echo["Publication"] = df_echo["Publication"].dt.date
df_echo = df_echo.loc[df_echo["Publication"]==now]

In [16]:
df_echo["Title"].iloc[0]
#df_echo["Description"].iloc[0]

"Coup d'envoi à la création de la plus grande banque espagnole"

In [17]:
book = epub.EpubBook()
book.set_title('Test Echo')
chapter_list = []

# Create chapter
def chapter(title, filename, content):
    chapter = epub.EpubHtml(title=title, file_name = filename, lang='fr')
    chapter.content = content
    return chapter

for i in range(0,5):
    chapter_list.append(chapter(df_echo["Title"].iloc[i],'chap_0'+ str(i) + '.xhtml',df_echo["Description"].iloc[i]))  

for chapter in chapter_list:
    book.add_item(chapter)

# Define CSS style
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

# Add CSS file
book.add_item(nav_css)

book.toc = (
    (
        epub.Section("Partie 1"),
        tuple(chapter_list)),
           (epub.Section("Partie 2"),
    tuple(chapter_list)),
           )
                      
# add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# Basic spine
for chapter in chapter_list:
    malist.append(chapter)

book.spine = malist

# write to the file
epub.write_epub('test.epub', book, {})

['nav', <ebooklib.epub.EpubHtml object at 0x0000027790618190>, <ebooklib.epub.EpubHtml object at 0x0000027790618430>, <ebooklib.epub.EpubHtml object at 0x00000277906182B0>, <ebooklib.epub.EpubHtml object at 0x00000277906182E0>, <ebooklib.epub.EpubHtml object at 0x0000027790618550>]


In [32]:
df_echo["Label"].value_counts()
label = df_echo["Label"].sort_values()
label = label.drop_duplicates()


In [55]:
print(df_echo.loc[0])

KeyError: 0

In [53]:
# Etape 1 : pour chaque section création des chapitres
chapter_list = []
for i, val in enumerate(df_echo.iterrows()):
    print(df_echo.at[0, "Label"])
    if df_echo.at[i, "Label"] == "Monde":
        chapter_list.append(chapter(df_echo.at[i,"Title"],'chap_0'+ str(i) + '.xhtml',df_echo.at[i, "Description"]))


KeyError: 0